In [1]:
##################################################################
#  Цель - отсканировать существующие препринты на сайте Банка России 
#     и собрать из этих них информацию о ключевых словах
# 1) Нужно научиться сканировать сайты и открывать пдф-файлы на этом сайте
# 2) Нужно научиться извлекать текст из этих pdf-файлов по заданному правилу
# 3) Нужно извлеченный текст соотнести с названием конкретного исследования в виде таблицы
# 4) Нужно экспортировать полученные данные в эксель формат
#
# Сайт 1: https://www.cbr.ru/ec_research/ser/
# Сайт 2: https://www.cbr.ru/dkp/system_p/
#################################################################

In [2]:
# библиотеки
import requests
import lxml
from bs4 import BeautifulSoup

In [3]:
# целевые сайты
url1 = 'https://www.cbr.ru/ec_research/ser/'
url2 = 'https://www.cbr.ru/dkp/system_p/research_notes/#a_119347'

r1 = requests.get(url1)
r2 = requests.get(url2)

# чтобы начать парсить, нужно преобразовать информацию с сайта в объект "Soup"
soup1 = BeautifulSoup(r1.text, 'html')
soup2 = BeautifulSoup(r2.text, 'html')

In [4]:
# находим все ссылки на исследования, которые расположены на сайте номер 1
base1 = soup1.find_all('a', class_ = 'referenceable')


In [5]:
# находим все ссылки на исследования, которые расположены на сайте номер 2
base2 = soup2.find_all('a')

In [6]:
# создаем словарь
research_data = []

In [7]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 1
for link in base1:
    # извлекаем название и ссылку
    title_span = link.find('span', class_='document-regular_name_visible')
    if title_span:
        title = title_span.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [8]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 1
for link in base2:
    # извлекаем значение атрибута href из ссылки
    href = link.get('href', '')
    if href and href.startswith('/StaticHtml') or href.startswith('/Content') :
        title = link.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [9]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/ec_research/ser/wp_155/

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/ec_research/ser/wp_154/

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/ec_research/ser/wp_153/

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/ec_research/ser/wp_152/

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/ec_research/ser/wp_151/

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/ec_research/ser/wp_150/

7. Декомпозиция индекса потребительских цен на 

In [10]:
# Теперь есть список статей и ссылки на них. Требуется из всех ссылок извлечь те, которые ведут на сайт и заменить их на ссылки на pdf файлы

In [11]:
def replace_with_pdf(articles_list):
    # данная функция заменяет ссылки на статьи на прямые ссылки на pdf-файлы
    results = articles_list.copy() # финальный словарь, в который будут сохраняться финальные ссылки на pdf

    for i, article in enumerate(results):
        current_link = article['link'] # получаем текущую ссылку на статью
        print(f"Обрабатываем: {current_link}")

        if current_link.lower().endswith('.pdf'):
            print("Уже PDF, пропускаем")
            continue
        
        else:
            print("Уже исправляю ссылку на PDF")
            urltest = current_link
            rtest = requests.get(urltest)
            souptest = BeautifulSoup(rtest.text, 'html.parser')
            basetest = souptest.find_all('a')

            for i, link in enumerate(basetest, 1):
                href = link.get('href')
                if href and href.endswith('.pdf'):
                    if href.startswith('/'):
                        hrefpdf = f"https://www.cbr.ru{href}"
                        article['link'] = hrefpdf
                    elif href.startswith('http'):
                        hrefpdf = href
                        article['link'] = hrefpdf
                    else:
                        hrefpdf = f"https://www.cbr.ru/{href}"
                        article['link'] = hrefpdf

    return results

research_data_pdf = replace_with_pdf(research_data)

Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_155/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_154/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_153/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_152/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_151/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_150/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_149/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_148/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_147/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_146/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_145/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_144/
Уже испр

In [12]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data_pdf, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/StaticHtml/File/180764/wp_155.pdf

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/StaticHtml/File/180505/wp_154.pdf

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/StaticHtml/File/178855/wp_153.pdf

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/StaticHtml/File/178678/wp_152.pdf

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/StaticHtml/File/177177/wp_151.pdf

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/StaticHtml/File/176714

## Извлекаем ключевые слова из PDF

In [13]:
import PyPDF2
from io import BytesIO
import re

### *Напишем функцию, которая извлекает текст из pdf-файла*

In [ ]:
def extract_text_from_pdf(pdf_link: str) -> str:
    response = requests.get(pdf_link)
    response.raise_for_status()

    pdf_data = BytesIO(response.content)

    reader = PyPDF2.PdfReader(pdf_data, strict=False)
    pdf_text = []

    for page in reader.pages:
        content = page.extract_text()
        pdf_text.append(content)
        
    return pdf_text



### *Напишем функцию, которая очищает текст*

In [ ]:
def clean_keywords_text(text: str) -> str: # данная функция очищает и восстанавливает текст
    combined_text = " ".join(text) # объединяем все страницы вместе
    lines = combined_text.split('\n') # разделяем текст на строки

    cleaned_lines = []
    for line in lines:
            # Убираем пробелы в начале и конце строки
            line = line.strip()
            cleaned_lines.append(line)

    full_text = ' '.join(cleaned_lines)

    full_text = re.sub(r'(\w+)-\s+', r'\1', full_text)
    full_text = re.sub(r'\s+', ' ', full_text)
    full_text = re.sub(r'\s+:', ':', full_text) # убираем пробел перед двоеточием
    full_text = re.sub(r':(\w)', r': \1', full_text) # добавляем пробел после двоеточия, если его нет

    return full_text.strip()

СЕРИЯ ДОКЛАДОВ ОБ ЭКОНОМИЧЕСКИХ ИССЛЕДОВАНИЯХ Динамика потенциального ВВП России после нефтяного шока: роль сильно го изменени я относительных цен и структурны х жесткост ей №6 / август 2015 г. Андрей Синяков * Агустин Ройтман Сергей Селезнев 2 Динамика потенциального ВВП России после нефтяного шока: роль сильного изменения относительных цен и структурны х жесткостей Серия докладов об экономических исследованиях Агустин Ройтман Международный Валютный Ф онд Андрей Синяков , * - автор , ответственный за переписку ( corresponding author ) Банк России . Email: Sinyakov AA@cbr.ru Сергей Селезнев Email: SeleznevSM @cbr.ru Авторы выражают благодарность профессору Carlos A. Vegh , Ph.D., за код программ ы из учебника Carlos A. Vegh (2013) , ставший отправной точкой нашей работы по моделированию экономики России . Авторы выражают благодарность Владимир у Аркадьевичу Бессонов у, к.ф.-м.н., Илье Борисовичу Воскобойникову , Ph.D., Алексе ю Евгеньевичу Девятов у, Ph.D., Серге ю Егиев у за критику, 

### *Напишем функцию, которая ищет в тексте ключевые слова*

In [ ]:
def extract_keywords_intelligent(text: str) -> str: # извлечение ключевых слов с учетом различных форматов:
                                                               # - останавливается при встрече слова с заглавной буквы после запятых
                                                               # - учитывает точки как разделители
                                                               # - обрабатывает различные форматы JEL классификации

   # варианты паттернов для поиска начала ключевых слов
    patterns = [
        r'Ключевые слова:\s*',
        r'Keywords:\s*',
        r'Key words:\s*',
        r'KEYWORDS:\s*',
        r'KEY WORDS:\s*',
        r'Ключевые слова\s*',  # Без двоеточия
        r'Keywords\s*',        # Без двоеточия
        r'Key words\s*',       # Без двоеточия
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start_pos = match.end()
            break

    # Находим позицию начала ключевых слов
    keywords_text = text[start_pos:]
    
    # Разбиваем текст на токены для анализа
    tokens = []
    current_token = ""
    
    # Проходим по каждому символу для анализа структуры
    for char in keywords_text:
        if char in [':', '.']:  # Сначала проверяем точку и двоеточие (если они есть, то значит ключевые слова 100% уже перечислены)
            if current_token:
                tokens.append(current_token)  # Добавляем последнее слово
            break  # Выходим из цикла

        elif char in ['ь', 'ы', 'ъ']:
            if current_token:
                tokens.append(current_token)
                current_token = ""
            tokens.append(char)

        elif char in [',', ';'] or char.isspace():
            if current_token:
                tokens.append(current_token)
                current_token = ""
            tokens.append(char)
        else:
            current_token += char
    
    return tokens


    # Собираем ключевые слова до условия остановки
    result_tokens = []
    stop = False
    prev_comma = False  # Флаг, что предыдущий токен был запятой
    
    for i, token in enumerate(tokens):
        if stop:
            break
        
        if (token[0].isupper() if token else False) and prev_comma:
            # Слово с заглавной буквы после запятой - вероятно начало нового раздела
            # Проверяем, не является ли это частью ключевого слова (например, "ВВП")
            if len(token) > 1 and not any(c.islower() for c in token[1:]):
                # Если все остальные буквы заглавные (акроним), то продолжаем
                result_tokens.append(token)
            else:
                # Останавливаемся на слове с заглавной буквы после запятой
                break
                
        elif token.lower().startswith('jel') or token.lower().startswith('j.e.l'):
            # начало JEL классификации
            break
            
        else:
            result_tokens.append(token)
        
        # Обновляем флаг предыдущей запятой
        prev_comma = (token == ',')
    
    # Собираем результат
    keywords_result = "".join(result_tokens).strip()
    
    # Очищаем результат от лишних символов в конце
    keywords_result = re.sub(r'[,\s]+$', '', keywords_result)  # Убираем запятые и пробелы в конце
    keywords_result = re.sub(r'\s+,\s*', ', ', keywords_result)  # Нормализуем запятые
    
    return keywords_result

In [ ]:
extracted_text = extract_text_from_pdf('https://www.cbr.ru/Content/Document/File/16735/wps_6.pdf')


In [32]:
clean_text = clean_keywords_text(extracted_text)

In [48]:
keywords = extract_keywords_intelligent(clean_text)
print(keywords)

['потенциальный', ' ', 'ВВП', ',', ' ', 'ослабление', ' ', 'реального', ' ', 'валютного', ' ', 'курса', ' ', ',', ' ', 'структурная', ' ', 'перестройка', ',', ' ', 'жесткости', ' ', 'перемещения', ' ', 'ресурсов', ' ', 'между', ' ', 'секторами', ',', ' ', 'нефтяной', ' ', 'шок', ',', ' ', 'совокупная', ' ', 'факторная', ' ', 'производител', ' ', 'ьность']
